In [ ]:
%pip install -e ..
%restart_python

In [ ]:
import json
import requests
import sys
from loguru import logger
from pyspark.sql import SparkSession

from pyspark.dbutils import DBUtils
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

In [ ]:


workspace_url = dbutils.widgets.get("workspace_url")
endpoint_name = dbutils.widgets.get("endpoint_name")
# token = dbutils.secrets.get("mlops", "databricks-pat")

payload_path = "../tests/battery_test.json"

with open(payload_path) as f:
    payload = json.load(f)

# url = f"https://{workspace_url}/serving-endpoints/{endpoint_name}/invocations"

# headers = {
#     "Authorization": f"Bearer {token}",
#     "Content-Type": "application/json"
# }
headers = {
    "Authorization": f"Bearer {dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()}",
    "Content-Type": "application/json",
}

resp = requests.post(endpoint_name, headers=headers, json=payload, timeout=30)

logger.info("Status: {}", resp.status_code)
logger.info("Response: {}", resp.text)

if resp.status_code != 200:
    raise RuntimeError("❌ Serving smoke test failed")

logger.info("✅ Serving smoke test passed")
